In [6]:
# Importar las librerías necesarias
import pandas as pd

# Cargar los datos de un archivo CSV

df_superstore = pd.read_csv('C:/Users/ljgov/Documents/LABORATORIA/PROYECTO_ETL/superstore.csv')


In [7]:

# Ejemplo de datos iniciales
data = {
    'category': ['Furniture', 'Office Supplies', 'Technology', 'furniture'],
    'city': ['Los Angeles', 'Los Angeles', 'los angeles', 'Los Angeles'],
    'country': ['United States', 'US', 'U.S.', 'United States'],
    'customer_ID': ['LS-172304', 'MV-174584', 'CS-123104', 'AP-109154'],
    'order_priority': ['High', 'Medium', 'Low', 'medium'],
    'market': ['US', 'US', 'US', 'US'],
    'order_date': ['7/01/2011 0:00:00', '8/01/2011 0:00:00', '9/01/2011 0:00:00', '10/01/2011 0:00:00'],
    'region': ['West', 'West', 'west', 'West'],
    'ship_date': ['9/01/2011 0:00:00', '10/01/2011 0:00:00', '11/01/2011 0:00:00', '12/01/2011 0:00:00']
}

df = pd.DataFrame(data)

# 1. Estandarización de textos
df['category'] = df['category'].str.title()  # Establecer formato título (ej. 'Furniture')
df['city'] = df['city'].str.title()  # Convertir el nombre de la ciudad en formato título
df['country'] = df['country'].str.upper()  # Convertir el nombre del país a mayúsculas
df['order_priority'] = df['order_priority'].str.capitalize()  # Convertir a formato título
df['region'] = df['region'].str.title()  # Convertir la región en formato título

# 2. Corrección de errores tipográficos y normalización
# Normalizar la columna 'country'
df['country'] = df['country'].replace(['US', 'U.S.'], 'UNITED STATES')

# Normalizar la columna 'order_priority'
df['order_priority'] = df['order_priority'].replace({
    'Critical': 4,
    'High': 3, 
    'Medium': 2, 
    'Low': 1
})

# 3. Codificación de variables categóricas
# Normalizar 'category'
category_mapping = {
    'Furniture': 1,
    'Office Supplies': 2,
    'Technology': 3
}
df['category_code'] = df['category'].map(category_mapping)

# 4. Conversión de las columnas de fecha al formato de fecha estándar
df['order_date'] = pd.to_datetime(df['order_date'], format='%d/%m/%Y %H:%M:%S')
df['ship_date'] = pd.to_datetime(df['ship_date'], format='%d/%m/%Y %H:%M:%S')

# 5. Si la hora es irrelevante, puedes eliminarla extrayendo solo la parte de la fecha
df['order_date'] = df['order_date'].dt.date
df['ship_date'] = df['ship_date'].dt.date

# 6. Extraer componentes de la fecha
df['order_year'] = pd.DatetimeIndex(df['order_date']).year
df['order_month'] = pd.DatetimeIndex(df['order_date']).month
df['order_day'] = pd.DatetimeIndex(df['order_date']).day

df['ship_year'] = pd.DatetimeIndex(df['ship_date']).year
df['ship_month'] = pd.DatetimeIndex(df['ship_date']).month
df['ship_day'] = pd.DatetimeIndex(df['ship_date']).day


# 7. Verificación de resultados
print(df)

# Si necesitas guardar el archivo nuevamente como CSV
# df.to_csv('ruta_al_archivo_normalizado.csv', index=False)


          category         city        country customer_ID  order_priority  \
0        Furniture  Los Angeles  UNITED STATES   LS-172304               3   
1  Office Supplies  Los Angeles  UNITED STATES   MV-174584               2   
2       Technology  Los Angeles  UNITED STATES   CS-123104               1   
3        Furniture  Los Angeles  UNITED STATES   AP-109154               2   

  market  order_date region   ship_date  category_code  order_year  \
0     US  2011-01-07   West  2011-01-09              1        2011   
1     US  2011-01-08   West  2011-01-10              2        2011   
2     US  2011-01-09   West  2011-01-11              3        2011   
3     US  2011-01-10   West  2011-01-12              1        2011   

   order_month  order_day  ship_year  ship_month  ship_day  
0            1          7       2011           1         9  
1            1          8       2011           1        10  
2            1          9       2011           1        11  
3            1

C:\Users\ljgov\AppData\Local\Temp\ipykernel_25420\1767030797.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['order_priority'] = df['order_priority'].replace({
